In [ ]:
%%bash
#count number of reads per library (prior to amplicon demultiplexing)
SAMPLE_DIR=Samples
COUNT=Results/n_reads_per_library.tsv

{ echo -e "Samples\tFragmentsNumber"; for f in `find Samples -name "*R1_001.fastq.gz"`; do zcat "$f" | echo -e "$(basename `dirname $f | cut -d - -f 1`)\t$(( `wc -l`/4 ))"; done } > $COUNT

### 1. demultiplex amplicons

In [ ]:
%%bash

mkdir -p Results/Amplicons_demultiplexed

for f in Samples/* 
    do 
    f=$(basename $f)
    cutadapt \
    -g file:qiaseq_primers_fwd-x.fa \
    -G file:qiaseq_primers_rev-x.fa \
    --pair-adapters --no-indels -e 0.1 \
    --suffix ':region={name}' \
    -o Results/Amplicons_demultiplexed/${f}_{name}_R1.fastq \
    -p Results/Amplicons_demultiplexed/${f}_{name}_R2.fastq Samples/${f}*/*R1* Samples/${f}*/*R2*
    done

In [ ]:
%%bash 
#move files from each region to a particular directory

mkdir -p Results/Amplicons_demultiplexed/V1V2
mkdir -p Results/Amplicons_demultiplexed/V2V3
mkdir -p Results/Amplicons_demultiplexed/V3V4
mkdir -p Results/Amplicons_demultiplexed/V4V5
mkdir -p Results/Amplicons_demultiplexed/V5V7
mkdir -p Results/Amplicons_demultiplexed/V7V9
mkdir -p Results/Amplicons_demultiplexed/ITS
mkdir -p Results/Amplicons_demultiplexed/unknown

mv Results/Amplicons_demultiplexed/*_V1V2* Results/Amplicons_demultiplexed/V1V2
mv Results/Amplicons_demultiplexed/*_V2V3* Results/Amplicons_demultiplexed/V2V3
mv Results/Amplicons_demultiplexed/*_V3V4* Results/Amplicons_demultiplexed/V3V4
mv Results/Amplicons_demultiplexed/*_V4V5* Results/Amplicons_demultiplexed/V4V5
mv Results/Amplicons_demultiplexed/*_V5V7* Results/Amplicons_demultiplexed/V5V7
mv Results/Amplicons_demultiplexed/*_V7V9* Results/Amplicons_demultiplexed/V7V9
mv Results/Amplicons_demultiplexed/*_ITS* Results/Amplicons_demultiplexed/ITS
mv Results/Amplicons_demultiplexed/*_unknown* Results/Amplicons_demultiplexed/unknown

In [ ]:
%%bash
#calculate read length for demultiplexed samples

mkdir -p Results/Read_length

for f in Results/Amplicons_demultiplexed/*/*
do
g=$(basename $f)
cat $f| awk '{if(NR%4==2) print length($1)}' | sort -n | uniq -c > Results/Read_length/${g}_read_length.txt
done

In [ ]:
%%bash
#calculate read length for all samples from a given region
#data generated here are going to contain a sum for each fastq
#that is, the files are being concatenated instead of summed up
#later I will sum properly using R

for f in Results/Amplicons_demultiplexed/*
do
region=$(basename $f)
for g in Results/Read_length/*${region}_R1*
do
cat $g >> Results/Read_length/${region}_R1_read_length.txt
done
for g in Results/Read_length/*${region}_R2*
do
cat $g >> Results/Read_length/${region}_R2_read_length.txt
done
done

In [ ]:
%%bash
#renaming files

for f in Results/Amplicons_demultiplexed/*
do
region=$(basename $f)
for g in Results/Amplicons_demultiplexed/${region}/*
do
g=$(basename $g)
mv ${f}/${g} ${f}/${g//\ds*_/}
done
done

#### generate artifacts

In [ ]:
%%bash
#a manifest indicating sequences loc must be prepared previously

#programatically generating from a generic manifest a manifest for each amplicon
for f in Results/Amplicons_demultiplexed/*
do
region=$(basename $f)
sed "s/region/$region/g" amplicons_demux_manifest > amplicons_demux_manifest_${region}
done

#generating qiime artifacts for each amplicon
for f in Results/Amplicons_demultiplexed/*
do
region=$(basename $f)
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path amplicons_demux_manifest_${region} \
  --input-format PairedEndFastqManifestPhred33V2 \
  --output-path Results/Amplicons_demultiplexed/${region}_demux-paired.qza
done

In [ ]:
%%bash
#summarize artifacts

for f in Results/Amplicons_demultiplexed/*qza
do
f=$(basename $f)
f=${f%.qza}
qiime demux summarize \
  --i-data Results/Amplicons_demultiplexed/${f}.qza \
  --o-visualization Results/Amplicons_demultiplexed/${f}.qzv
done

### 2. dada2

In [ ]:
%%bash
mkdir Results/Dadaed

trunc_len_f=( 0 0 0 0 0 0 0 )
trunc_len_r=( 0 241 234 236 210 213 226 ) #array with different truncation parametes for R1 and R2 for each amplicon
i=0                                 #values were chosen so that everything that comes after (3') the first instance of median quality<30 is disconsidered
                                #order in the array: ITS, followed by 16S amplicons in the usual order
i=0                                
for f in Results/Amplicons_demultiplexed/[IV]*qza
do
region=$(basename $f)
qiime dada2 denoise-paired \
    --i-demultiplexed-seqs Results/Amplicons_demultiplexed/${region} \
    --p-trunc-len-f ${trunc_len_f[$i]} \
    --p-trunc-len-r ${trunc_len_r[$i]} \
    --o-table Results/Dadaed/${region}_table.qza \
    --o-representative-sequences Results/Dadaed/${region}_rep-seqs.qza \
    --o-denoising-stats Results/Dadaed/${region}_stats.qza
i=$i+1
done

In [ ]:
%%bash
#summarize dada2 feature-tables

for f in Results/Dadaed/*table.qza
do
f=$(basename $f)
f=${f%.qza}
qiime feature-table summarize \
  --i-table Results/Dadaed/${f}.qza \
  --o-visualization Results/Dadaed/${f}.qzv
done

In [ ]:
%%bash
#fixing file names in dadaed repository

for f in Results/Dadaed/*
do
mv $f ${f//\de*_/}
done

### 3. ref-based chimera removal

In [ ]:
%%bash
#identify chimeric seqs

mkdir Results/Chimeras

for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime vsearch uchime-ref \
  --i-sequences Results/Dadaed/${f}_rep-seqs.qza \
  --i-table Results/Dadaed/${f}_table.qza \
  --i-reference-sequences /home/vitorheidrich/SILVA/silva-138-99-seqs.qza \
  --o-chimeras Results/Chimeras/${f}_chimeric-seqs.qza \
  --o-nonchimeras Results/Chimeras/${f}_nonchimeric-seqs.qza \
  --o-stats Results/Chimeras/${f}_chimera_stats.qza
done

In [ ]:
%%bash
#remove chimeric seqs from dadaed data

mkdir Results/Dadaed-chimremoved

for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime feature-table filter-features \
  --i-table Results/Dadaed/${f}_table.qza \
  --m-metadata-file Results/Chimeras/${f}_nonchimeric-seqs.qza \
  --o-filtered-table Results/Dadaed-chimremoved/${f}_table.qza
  
qiime feature-table filter-seqs \
  --i-data Results/Dadaed/${f}_rep-seqs.qza \
  --m-metadata-file Results/Chimeras/${f}_nonchimeric-seqs.qza \
  --o-filtered-data Results/Dadaed-chimremoved/${f}_rep-seqs.qza
  
qiime feature-table summarize \
  --i-table Results/Dadaed-chimremoved/${f}_table.qza \
  --o-visualization Results/Dadaed-chimremoved/${f}_table.qzv
done

### 4. taxonomic assignment

In [ ]:
%%bash

#getting silva data

qiime rescript get-silva-data \
    --p-version '138' \
    --p-target 'SSURef_NR99' \
    --p-include-species-labels \
    --o-silva-sequences /home/vitorheidrich/SILVA/silva-138-ssu-nr99-seqs.qza \
    --o-silva-taxonomy /home/vitorheidrich/SILVA/silva-138-ssu-nr99-tax.qza

In [ ]:
%%bash

#removing low quality ref-seqs

qiime rescript cull-seqs \
    --i-sequences /home/vitorheidrich/SILVA/silva-138-ssu-nr99-seqs.qza \
    --o-clean-sequences /home/vitorheidrich/SILVA/silva-138-ssu-nr99-seqs-cleaned.qza

In [ ]:
%%bash

#dereplicating identical seqs

qiime rescript dereplicate \
    --i-sequences /home/vitorheidrich/SILVA/silva-138-ssu-nr99-seqs-cleaned.qza  \
    --i-taxa /home/vitorheidrich/SILVA/silva-138-ssu-nr99-tax.qza \
    --p-rank-handles 'silva' \
    --p-mode 'uniq' \
    --o-dereplicated-sequences /home/vitorheidrich/SILVA/silva-138-ssu-nr99-seqs-derep-uniq.qza \
    --o-dereplicated-taxa /home/vitorheidrich/SILVA/silva-138-ssu-nr99-tax-derep-uniq.qza

In [ ]:
%%bash

#extracting specific regions from ref-seqs for each amplicon

f_primer=( AGRGTTTGATYMTGGCTC GGCGNACGGGTGAGTAA CCTACGGGNGGCWGCAG GTGYCAGCMGCCGCGGTAA GGATTAGATACCCBRGTAGTC YAACGAGCGMRACCC )
r_primer=( CTGCTGCCTYCCGTA WTTACCGCGGCTGCTGG GACTACHVGGGTATCTAATCC CCGYCAATTYMTTTRAGTTT ACGTCRTCCCCDCCTTCCTC TACGGYTACCTTGTTAYGACTT )
i=0

for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime feature-classifier extract-reads \
    --i-sequences /home/vitorheidrich/SILVA/silva-138-ssu-nr99-seqs-derep-uniq.qza \
    --p-f-primer ${f_primer[$i]} \
    --p-r-primer ${r_primer[$i]} \
    --p-min-length 100 \
    --p-max-length 600 \
    --p-n-jobs 2 \
    --p-read-orientation 'forward' \
    --o-reads /home/vitorheidrich/SILVA/silva-138-ssu-nr99-seqs_${f}.qza
i=$i+1
done

In [ ]:
%%bash
#rodado no ssh
#training feature classifier

for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads /home/vitorheidrich/SILVA/silva-138-ssu-nr99-seqs_${f}.qza \
  --i-reference-taxonomy /home/vitorheidrich/SILVA/silva-138-ssu-nr99-tax-derep-uniq.qza \
  --o-classifier /home/vitorheidrich/SILVA/${f}_classifier.qza
done

In [ ]:
%%bash
#rodado no ssh
#performing taxonomic assignment

mkdir Results/Taxonomy

for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime feature-classifier classify-sklearn \
  --i-classifier /home/vitorheidrich/SILVA/${f}_classifier.qza \
  --i-reads Results/Dadaed-chimremoved/${f}_rep-seqs.qza \
  --o-classification Results/Taxonomy/${f}_taxonomy.qza

qiime metadata tabulate \
  --m-input-file Results/Taxonomy/${f}_taxonomy.qza \
  --o-visualization Results/Taxonomy/${f}_taxonomy.qzv
done

### 5. remove non-bacterial seqs 

In [ ]:
%%bash

#remove reads unassigned (including unassigned bacteria) or non-bacterial (archaea, chloroplast, mitochondria, etc...)

#mkdir Results/Dadaed-chimremoved-nonbactremoved

NONBACT=Unassigned,Chloroplast,Mitochondria,Eukaryota,Archaea

for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime taxa filter-seqs \
  --i-sequences Results/Dadaed-chimremoved/${f}_rep-seqs.qza \
  --i-taxonomy Results/Taxonomy/${f}_taxonomy.qza \
  --p-exclude $NONBACT \
  --o-filtered-sequences Results/Dadaed-chimremoved-nonbactremoved/${f}_rep-seqs.qza

qiime taxa filter-table\
  --i-table Results/Dadaed-chimremoved/${f}_table.qza \
  --i-taxonomy Results/Taxonomy/${f}_taxonomy.qza \
  --p-exclude $NONBACT \
  --o-filtered-table Results/Dadaed-chimremoved-nonbactremoved/${f}_table.qza
  
qiime taxa filter-seqs \
  --i-sequences Results/Dadaed-chimremoved-nonbactremoved/${f}_rep-seqs.qza \
  --i-taxonomy Results/Taxonomy/${f}_taxonomy.qza \
  --p-exclude "d__Bacteria" \
  --p-mode exact \
  --o-filtered-sequences Results/Dadaed-chimremoved-nonbactremoved/${f}_rep-seqs.qza

qiime taxa filter-table\
  --i-table Results/Dadaed-chimremoved-nonbactremoved/${f}_table.qza \
  --i-taxonomy Results/Taxonomy/${f}_taxonomy.qza \
  --p-exclude "d__Bacteria" \
  --p-mode exact \
  --o-filtered-table Results/Dadaed-chimremoved-nonbactremoved/${f}_table.qza

qiime feature-table summarize \
  --i-table Results/Dadaed-chimremoved-nonbactremoved/${f}_table.qza \
  --o-visualization Results/Dadaed-chimremoved-nonbactremoved/${f}_table.qzv
  
done

In [ ]:
%%bash

#for V5V7, the most abundant ASV in SmartControl was assigned to Enterobacterales, 
#that is an artifact; for other amplicons the most abundant ASV was SmartControl was identified as unassigned bacteria
#so we will filter this manually below

qiime feature-table filter-seqs \
  --i-data Results/Dadaed-chimremoved-nonbactremoved/V5V7_rep-seqs.qza \
  --m-metadata-file Results/Taxonomy/V5V7_taxonomy.qza \
  --p-where "[Feature ID]='d961cc92ebafe55bd35d1f411f10babb'" \
  --p-exclude-ids \
  --o-filtered-data Results/Dadaed-chimremoved-nonbactremoved/V5V7_rep-seqs.qza

qiime feature-table  filter-features\
  --i-table Results/Dadaed-chimremoved-nonbactremoved/V5V7_table.qza \
  --m-metadata-file Results/Taxonomy/V5V7_taxonomy.qza \
  --p-where "[Feature ID]='d961cc92ebafe55bd35d1f411f10babb'" \
  --p-exclude-ids \
  --o-filtered-table Results/Dadaed-chimremoved-nonbactremoved/V5V7_table.qza

qiime feature-table summarize \
  --i-table Results/Dadaed-chimremoved-nonbactremoved/V5V7_table.qza \
  --o-visualization Results/Dadaed-chimremoved-nonbactremoved/V5V7_table.qzv

### 6. remove contaminants

In [ ]:
%%bash

#inspect contaminants of negative controls

#mkdir Results/Taxa-barplots

for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime taxa barplot \
  --i-table Results/Dadaed-chimremoved-nonbactremoved/${f}_table.qza \
  --i-taxonomy Results/Taxonomy/${f}_taxonomy.qza \
  --o-visualization Results/Taxa-barplots/${f}_taxa-barplot_contam-inspect.qzv
done

In [ ]:
#the contaminants will be identified with the help of the decontam R package elsewhere

In [8]:
%%bash

#remove contaminants as identified by decontam and confirmed by manual curation

#mkdir Results/Dadaed-chimremoved-nonbactremoved-contamremoved

CONTAM=Pelomonas,Mycoplasma_wenyonii,Candidatus_Obscuribacter

for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime taxa filter-seqs \
  --i-sequences Results/Dadaed-chimremoved-nonbactremoved/${f}_rep-seqs.qza \
  --i-taxonomy Results/Taxonomy/${f}_taxonomy.qza \
  --p-exclude $CONTAM \
  --o-filtered-sequences Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_rep-seqs.qza

qiime taxa filter-table\
  --i-table Results/Dadaed-chimremoved-nonbactremoved/${f}_table.qza \
  --i-taxonomy Results/Taxonomy/${f}_taxonomy.qza \
  --p-exclude $CONTAM \
  --o-filtered-table Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_table.qza

qiime feature-table summarize \
  --i-table Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_table.qza \
  --o-visualization Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_table.qzv

done

Saved FeatureData[Sequence] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V1V2_rep-seqs.qza
Saved FeatureTable[Frequency] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V1V2_table.qza
Saved Visualization to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V1V2_table.qzv
Saved FeatureData[Sequence] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V2V3_rep-seqs.qza
Saved FeatureTable[Frequency] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V2V3_table.qza
Saved Visualization to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V2V3_table.qzv
Saved FeatureData[Sequence] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V3V4_rep-seqs.qza
Saved FeatureTable[Frequency] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V3V4_table.qza
Saved Visualization to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V3V4_table.qzv
Saved FeatureData[Sequence] to: Results/Dadaed-chimremoved-nonbactremoved-contamre

In [9]:
%%bash

#remove control samples and ASVs exclusively appearing in controls

for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime feature-table filter-samples \
  --i-table Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_table.qza \
  --m-metadata-file Results/Dadaed-chimremoved-nonbactremoved/decontam-metadata.tsv \
  --p-where "[Sample_or_Control]='True Sample'" \
  --p-filter-empty-features \
  --o-filtered-table Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_table.qza

qiime feature-table filter-seqs \
  --i-data Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_rep-seqs.qza \
  --i-table Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_table.qza \
  --o-filtered-data Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_rep-seqs.qza

qiime feature-table summarize \
  --i-table Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_table.qza \
  --o-visualization Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_table.qzv

done

Saved FeatureTable[Frequency] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V1V2_table.qza
Saved FeatureData[Sequence] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V1V2_rep-seqs.qza
Saved Visualization to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V1V2_table.qzv
Saved FeatureTable[Frequency] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V2V3_table.qza
Saved FeatureData[Sequence] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V2V3_rep-seqs.qza
Saved Visualization to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V2V3_table.qzv
Saved FeatureTable[Frequency] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V3V4_table.qza
Saved FeatureData[Sequence] to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V3V4_rep-seqs.qza
Saved Visualization to: Results/Dadaed-chimremoved-nonbactremoved-contamremoved/V3V4_table.qzv
Saved FeatureTable[Frequency] to: Results/Dadaed-chimremoved-nonbactremoved-contam

### 7. generate tree 

In [10]:
%%bash

#generate phylogenetic trees

mkdir Results/Tree

for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do

qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_rep-seqs.qza \
    --o-alignment Results/Tree/${f}_aligned-seqs.qza \
    --o-masked-alignment Results/Tree/${f}_masked-alignment.qza \
    --o-tree Results/Tree/${f}_unrooted-tree.qza \
    --o-rooted-tree Results/Tree/${f}_rooted-tree.qza 

done

Saved FeatureData[AlignedSequence] to: Results/Tree/V1V2_aligned-seqs.qza
Saved FeatureData[AlignedSequence] to: Results/Tree/V1V2_masked-alignment.qza
Saved Phylogeny[Unrooted] to: Results/Tree/V1V2_unrooted-tree.qza
Saved Phylogeny[Rooted] to: Results/Tree/V1V2_rooted-tree.qza
Saved FeatureData[AlignedSequence] to: Results/Tree/V2V3_aligned-seqs.qza
Saved FeatureData[AlignedSequence] to: Results/Tree/V2V3_masked-alignment.qza
Saved Phylogeny[Unrooted] to: Results/Tree/V2V3_unrooted-tree.qza
Saved Phylogeny[Rooted] to: Results/Tree/V2V3_rooted-tree.qza
Saved FeatureData[AlignedSequence] to: Results/Tree/V3V4_aligned-seqs.qza
Saved FeatureData[AlignedSequence] to: Results/Tree/V3V4_masked-alignment.qza
Saved Phylogeny[Unrooted] to: Results/Tree/V3V4_unrooted-tree.qza
Saved Phylogeny[Rooted] to: Results/Tree/V3V4_rooted-tree.qza
Saved FeatureData[AlignedSequence] to: Results/Tree/V4V5_aligned-seqs.qza
Saved FeatureData[AlignedSequence] to: Results/Tree/V4V5_masked-alignment.qza
Saved Ph

mkdir: cannot create directory ‘Results/Tree’: File exists


### 8. sidle

In [ ]:
%%bash

#ran in server DONE

#prepare previously extracted regions

#there is a difference between the reference database used here and the one used previously to classify sequences (stored at /home/users/vheidrich/projects_current/Metagenomics/BCG-16S/SILVA)
#há uma diferença entre essas bases de dados e as geradas anteriormente para o feature-classifier (que estão guardadas no )
#the one used here was generated with RESCRIPt allowing a lower number of ambiguous bases
#specifically: rescript cull-seqs with max 3 ("--p-num-degenerates 4"), as Sidle recommends
#before we used rescript cull-seqs with max 4 ("--p-num-degenerates 5")

#Sidle-reconstructed datasets combining pairs of amplicons were built through an analogous pipeline (not shown for shortness)

mkdir Results/Sidle

f_primer=( AGRGTTTGATYMTGGCTC GGCGNACGGGTGAGTAA CCTACGGGNGGCWGCAG GTGYCAGCMGCCGCGGTAA GGATTAGATACCCBRGTAGTC YAACGAGCGMRACCC )
r_primer=( CTGCTGCCTYCCGTA WTTACCGCGGCTGCTGG GACTACHVGGGTATCTAATCC CCGYCAATTYMTTTRAGTTT ACGTCRTCCCCDCCTTCCTC TACGGYTACCTTGTTAYGACTT )
i=0
for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime sidle prepare-extracted-region \
    --i-sequences /home/vitorheidrich/SILVA/silva-138-ssu-nr99-seqs_${f}.qza \
    --p-region $f \
    --p-fwd-primer ${f_primer[$i]} \
    --p-rev-primer ${r_primer[$i]} \
    --p-trim-length 300 \
    --o-collapsed-kmers Results/Sidle/db_${f}_300nt-kmers.qza \
    --o-kmer-map Results/Sidle/db_${f}_300nt-map.qza
i=$i+1
done

In [11]:
%%bash

#trim reads post-hoc to the same length chosen for db trimming above
#this choice was based on the distribution of ASV lengths (analyzed in one of the R scripts)

for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime sidle trim-dada2-posthoc \
    --i-table Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_table.qza \
    --i-representative-sequences Results/Dadaed-chimremoved-nonbactremoved-contamremoved/${f}_rep-seqs.qza \
    --p-trim-length 300 \
    --o-trimmed-table Results/Sidle/${f}_table-300nt.qza \
    --o-trimmed-representative-sequences Results/Sidle/${f}_rep-seqs-300nt.qza

qiime feature-table summarize \
  --i-table Results/Sidle/${f}_table-300nt.qza\
  --o-visualization Results/Sidle/${f}_table-300nt.qzv
done

Saved FeatureTable[Frequency] to: Results/Sidle/V1V2_table-300nt.qza
Saved FeatureData[Sequence] to: Results/Sidle/V1V2_rep-seqs-300nt.qza
Saved Visualization to: Results/Sidle/V1V2_table-300nt.qzv
Saved FeatureTable[Frequency] to: Results/Sidle/V2V3_table-300nt.qza
Saved FeatureData[Sequence] to: Results/Sidle/V2V3_rep-seqs-300nt.qza
Saved Visualization to: Results/Sidle/V2V3_table-300nt.qzv
Saved FeatureTable[Frequency] to: Results/Sidle/V3V4_table-300nt.qza
Saved FeatureData[Sequence] to: Results/Sidle/V3V4_rep-seqs-300nt.qza
Saved Visualization to: Results/Sidle/V3V4_table-300nt.qzv
Saved FeatureTable[Frequency] to: Results/Sidle/V4V5_table-300nt.qza
Saved FeatureData[Sequence] to: Results/Sidle/V4V5_rep-seqs-300nt.qza
Saved Visualization to: Results/Sidle/V4V5_table-300nt.qzv
Saved FeatureTable[Frequency] to: Results/Sidle/V5V7_table-300nt.qza
Saved FeatureData[Sequence] to: Results/Sidle/V5V7_rep-seqs-300nt.qza
Saved Visualization to: Results/Sidle/V5V7_table-300nt.qzv
Saved Feat

In [ ]:
%%bash
#ran in server DONE
#align regional kmers
for f in {V1V2,V2V3,V3V4,V4V5,V5V7,V7V9}
do
qiime sidle align-regional-kmers \
    --i-kmers Results/Sidle/db_${f}_300nt-kmers.qza \
    --i-rep-seq Results/Sidle/${f}_rep-seqs-300nt.qza \
    --p-region $f \
    --p-n-workers 4 \
    --p-max-mismatch 5 \
    --o-regional-alignment Results/Sidle/${f}_align-map.qza
done

In [ ]:
%%bash
#ran in server DONE
#reconstruct database
qiime sidle reconstruct-database \
    --p-region V1V2 \
      --i-kmer-map Results/Sidle/db_V1V2_300nt-map.qza \
      --i-regional-alignment Results/Sidle/V1V2_align-map.qza \
    --p-region V2V3 \
      --i-kmer-map Results/Sidle/db_V2V3_300nt-map.qza \
      --i-regional-alignment Results/Sidle/V2V3_align-map.qza \
    --p-region V3V4 \
      --i-kmer-map Results/Sidle/db_V3V4_300nt-map.qza \
      --i-regional-alignment Results/Sidle/V3V4_align-map.qza \
    --p-region V4V5 \
      --i-kmer-map Results/Sidle/db_V4V5_300nt-map.qza \
      --i-regional-alignment Results/Sidle/V4V5_align-map.qza \
    --p-region V5V7 \
      --i-kmer-map Results/Sidle/db_V5V7_300nt-map.qza \
      --i-regional-alignment Results/Sidle/V5V7_align-map.qza \
    --p-region V7V9 \
      --i-kmer-map Results/Sidle/db_V7V9_300nt-map.qza \
      --i-regional-alignment Results/Sidle/V7V9_align-map.qza \
    --o-database-summary Results/Sidle/full_summary.qza \
    --o-database-map Results/Sidle/full_map.qza

In [ ]:
%%bash
#ran in server DONE
#reconstruct feature table
qiime sidle reconstruct-counts \
    --p-region V1V2 \
      --i-regional-alignment Results/Sidle/V1V2_align-map.qza \
      --i-regional-table Results/Sidle/V1V2_table-300nt.qza \
    --p-region V2V3 \
      --i-regional-alignment Results/Sidle/V2V3_align-map.qza \
      --i-regional-table Results/Sidle/V2V3_table-300nt.qza \
    --p-region V3V4 \
      --i-regional-alignment Results/Sidle/V3V4_align-map.qza \
      --i-regional-table Results/Sidle/V3V4_table-300nt.qza \
    --p-region V4V5 \
      --i-regional-alignment Results/Sidle/V4V5_align-map.qza \
      --i-regional-table Results/Sidle/V4V5_table-300nt.qza \
    --p-region V5V7 \
      --i-regional-alignment Results/Sidle/V5V7_align-map.qza \
      --i-regional-table Results/Sidle/V5V7_table-300nt.qza \
    --p-region V7V9 \
      --i-regional-alignment Results/Sidle/V7V9_align-map.qza \
      --i-regional-table Results/Sidle/V7V9_table-300nt.qza \
    --o-reconstructed-table Results/Sidle/full_table.qza \
    --p-min-counts 0 \
    --i-database-summary Results/Sidle/full_summary.qza \
    --i-database-map Results/Sidle/full_map.qza
    

qiime feature-table summarize \
  --i-table Results/Sidle/full_table.qza \
  --o-visualization Results/Sidle/full_table.qzv

In [1]:
%%bash
#ran in server DONE
#reconstruct taxonomy

qiime sidle reconstruct-taxonomy \
    --i-reconstruction-map Results/Sidle/full_map.qza \
    --i-taxonomy silva-138-ssu-nr99-tax-derep-uniq.qza \
    --p-database 'silva' \
    --p-define-missing 'ignore' \
    --o-reconstructed-taxonomy Results/Sidle/full_taxonomy.qza
 
qiime metadata tabulate \
    --m-input-file Results/Sidle/full_taxonomy.qza \
    --o-visualization Results/Sidle/full_taxonomy.qzv

In [ ]:
#%%bash

#NEVER USED

#reconstruct phylogenetic tree
#reconstruct consensus fragments that could not be resolved

#qiime sidle reconstruct-fragment-rep-seqs \
#    --p-region V1V2 \
#    --i-regional-alignment Results/Sidle/V1V2_align-map.qza \
#    --p-region V2V3 \
#    --i-regional-alignment Results/Sidle/V2V3_align-map.qza \
#    --p-region V3V4 \
#    --i-regional-alignment Results/Sidle/V3V4_align-map.qza \
#    --p-region V4V5 \
#    --i-regional-alignment Results/Sidle/V4V5_align-map.qza \
#    --p-region V5V7 \
#    --i-regional-alignment Results/Sidle/V5V7_align-map.qza \
#    --p-region V7V9 \
#    --i-regional-alignment Results/Sidle/V7V9_align-map.qza \
#    --i-reconstruction-map Results/Sidle/full_map.qza \
#    --i-reconstruction-summary Results/Sidle/full_summary.qza \
#    --i-aligned-sequences database/sidle-db-aligned-sequences.qza \ #que isso?????
#    --o-representative-fragments Results/Sidle/full-rep-seqs-fragments.qza

In [ ]:
#%%bash

#NEVER USED

#do the fragment insertion
#qiime fragment-insertion sepp \
#    --i-representative-sequences Results/Sidle/full-rep-seqs-fragments.qza \
#    --i-reference-database sepp-ref-silva-138.qza \ #this pre-built reference tree is not available
#    --o-tree Results/Sidle/full-tree.qza \          #i will have to either build it myself, wait for the v138
#    --o-placements Results/Sidle/full-placements.qza#sepp ref tree release or ignore phylogenetic trees